### Descrição do problema

A indústria musical busca constantemente entender os fatores que levam uma música ao sucesso. Neste trabalho, exploramos a possibilidade de prever a popularidade de canções através de análises de dados.</br></br>Utilizando um conjunto de dados do Kaggle (**https://www.kaggle.com/datasets/yasserh/song-popularity-dataset/data**), que contém informações sobre diversas métricas de áudio (energia, dançabilidade, intensidade, etc.), buscamos desenvolver modelos de machine learning capazes de identificar padrões e correlações entre essas características com o intuito de prever a popularidade das músicas.</br></br> 
Para a construção dos modelos, exploramos algoritmos como **Linear Regression Lasso**, **Linear Regression Ridge**, **Linear Regression Elastic Net**, **Decision Tree**, **Random Forest** e regressões polinomiais.</br>A avaliação dos modelos será realizada utilizando as seguintes métricas: **R²**, **MSE**, **RMSE**, **MAE** e **MAPE**. 

### Informações do dataset

- **song_name**: nome da música</br>
- **song_duration_ms**: duração da música em milissegundos</br>
- **acousticness**: Grau de presença de instrumentos acústicos na música</br>
- **danceability**: Quanto maior o valor, mais a música é adequada para dançar</br>
- **energy**: Grau de vigor e intensidade da música</br>
- **instrumentalness**: Grau de presença de vocais na música. Valores mais altos indicam músicas mais instrumentais</br>
- **key**: A tonalidade principal da música</br>
- **liveness**: Grau de presença de um público ao vivo na gravação. Valores mais altos indicam mais probabilidade de ser uma gravação ao vivo</br>
- **loudness**: Volume geral da música. Medido em decibéis (dB)</br>
- **audio_mode**: Indica se a música está em mono ou estéreo</br>
- **speechiness**: Grau de presença de fala na música</br>
- **tempo**: Velocidade da música, medida em batidas por minuto (BPM)</br>
- **time_signature**: A métrica da música, indicando o número de batidas por compasso</br>
- **audio_valence**: Positividade da música. Valores mais altos indicam músicas mais positivas</br>
- **song_popularity**: nível de popularidade da música, variando entre **0** (mínimo) e **100** (máximo).

### Observações importantes

Visando otimizar o aprendizado, os dados brutos foram submetidos a uma etapa de pré-processamento pelo instrutor do curso que incluiu a normalização dos atributos numéricos e a extração da coluna categórica `song_name`. Com isso, os dados já se encontram preparados para a aplicação dos algoritmos de machine learning utilizados nesse ensaio.

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

# 2.0 - Carregando os dados

In [3]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_val.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

In [4]:
X_train.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0.205673,0.092100,0.720,0.802,0.000000,0.090909,0.694,0.431778,1,0.0582,0.103876,0.8,0.723
1,-0.240409,0.737000,0.483,0.412,0.000000,0.636364,0.116,-0.262732,1,0.0402,1.711532,0.6,0.247
2,-0.125770,0.274000,0.671,0.565,0.000065,1.000000,0.370,0.013612,0,0.1600,1.009176,0.8,0.561
3,-0.580967,0.002340,0.704,0.529,0.874000,1.000000,0.370,-0.266382,0,0.0416,0.666173,0.8,0.507
4,-0.688566,0.000414,0.354,0.910,0.205000,0.090909,0.456,0.422914,1,0.0430,1.182630,0.8,0.362


## 2.1 - Variáveis preditoras: informações gerais

Nas células abaixo, mostramos informações gerais sobre os dados de **treino**, **validação** e **teste**.</br>Observamos que todas as variáveis preditoras são numéricas (formatos **int64**, **float64**), assim não há necessidade de transformar dados categóricos em numéricos.</br>Além disso, todos os conjuntos de dados estão completos, sem a necessidade de lidar com dados faltantes. A checagem preliminar dos dados mostrou que diversos atributos foram a priori rescalonados.</br>Lembramos que essa etapa é de suma importância para os algoritmos, como é o caso das Regressões lineares e Polinomiais. Caso contrário, alguns atributos poderiam ter um impacto muito maior do que outros no cálculo das métricas, traduzindo em valores de performance não confiáveis.</br>No contexto do presente ensaio, voltado para a exploração de diferentes combinações de hiperparâmetros sobre as métricas de performance, é razoável que as etapas de pré-processamento e separação de dados tenham sido realizadas de antemão pelo instrutor, dado que se trata de uma disciplina introdutória.

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  10547 non-null  float64
 1   acousticness      10547 non-null  float64
 2   danceability      10547 non-null  float64
 3   energy            10547 non-null  float64
 4   instrumentalness  10547 non-null  float64
 5   key               10547 non-null  float64
 6   liveness          10547 non-null  float64
 7   loudness          10547 non-null  float64
 8   audio_mode        10547 non-null  int64  
 9   speechiness       10547 non-null  float64
 10  tempo             10547 non-null  float64
 11  time_signature    10547 non-null  float64
 12  audio_valence     10547 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 1.0 MB


In [6]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  4521 non-null   float64
 1   acousticness      4521 non-null   float64
 2   danceability      4521 non-null   float64
 3   energy            4521 non-null   float64
 4   instrumentalness  4521 non-null   float64
 5   key               4521 non-null   float64
 6   liveness          4521 non-null   float64
 7   loudness          4521 non-null   float64
 8   audio_mode        4521 non-null   int64  
 9   speechiness       4521 non-null   float64
 10  tempo             4521 non-null   float64
 11  time_signature    4521 non-null   float64
 12  audio_valence     4521 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 459.3 KB


In [7]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3767 entries, 0 to 3766
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  3767 non-null   float64
 1   acousticness      3767 non-null   float64
 2   danceability      3767 non-null   float64
 3   energy            3767 non-null   float64
 4   instrumentalness  3767 non-null   float64
 5   key               3767 non-null   float64
 6   liveness          3767 non-null   float64
 7   loudness          3767 non-null   float64
 8   audio_mode        3767 non-null   int64  
 9   speechiness       3767 non-null   float64
 10  tempo             3767 non-null   float64
 11  time_signature    3767 non-null   float64
 12  audio_valence     3767 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 382.7 KB


## 2.2 - Variável resposta: informações gerais

In [8]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  10547 non-null  float64
dtypes: float64(1)
memory usage: 82.5 KB


In [9]:
y_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  4521 non-null   float64
dtypes: float64(1)
memory usage: 35.4 KB


In [10]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3767 entries, 0 to 3766
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  3767 non-null   float64
dtypes: float64(1)
memory usage: 29.6 KB
